In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

# from pyspark.sql import functions as F
# from pyspark.sql import types as T

from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.ml.linalg import Vectors, VectorUDT

In [4]:
a = np.round(np.random.rand(100), 2).tolist()

In [5]:
# Let us use python Array
data2 = [(1, a,),
         (1, a,),
         (2, a,),
         (2, a,)]

df = spark.createDataFrame(data2, ["id", "features"])

# Make it larger, up to 1024 rows 
for i in range(8):
    df = df.union(df)


df.show()
df.printSchema()
df.count()

+---+--------------------+
| id|            features|
+---+--------------------+
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
+---+--------------------+
only showing top 20 rows

root
 |-- id: long (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: double (containsNull = true)



1024

In [6]:
# Convert the python Array type to Dataframe array Type. 
convert_Array_udf = udf(lambda x: x, ArrayType(FloatType(), containsNull=False))


df = df.withColumn('features_array', convert_Array_udf('features')).drop('features')

df.show()
df.printSchema()

+---+--------------------+
| id|      features_array|
+---+--------------------+
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  1|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
|  2|[0.53, 0.74, 0.17...|
+---+--------------------+
only showing top 20 rows

root
 |-- id: long (nullable = true)
 |-- features_array: array (nullable = true)
 |    |-- element: float (containsNull = false)



In [7]:
# Sum Column-Wise
from pyspark.sql import functions as F

n = len(df.select('features_array').first()[0])

print(n)

resultDF = df.agg(F.array(*[F.sum(F.col("features_array")[i]) for i in range(n)]).alias("sum"))

resultDF.show(truncate=False)
# resultDF.show()

100
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------